In [83]:
import os
import subprocess
import warnings
from collections import Counter
from pathlib import Path
import numpy as np
import pandas as pd
from Bio import SeqIO
from pandas.core.common import SettingWithCopyWarning
import pysam

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)


# Data location

In [163]:
threads = 30
ont_reads = "ont_reads"
short_reads_1_location = "short_reads_1_location"
short_reads_2_location = "short_reads_2_location"
working_directory = "working_directory"
Path(working_directory).mkdir(parents=True, exist_ok=True)
# database locations
card_database = "card_database"#card_data/nucleotide_fasta_protein_homolog_model.fasta"
card_database_general = "card_database_general"#/card/card_database_v3.1.4.fasta"
blast_nt_database = "blast_nt_database"#blast/nt/nt"
mge_database = "mge_database"#/MGEs90.fasta"
centrifuge_database = "centrifuge_database"#centrifuge/abv"
mob_suite_database = "mob_suite_database"#mob_suite/databases"

In [137]:
ont_card_blast = os.path.join(working_directory, "ont_card_blast.out")
ont_arg_related_reads_list = os.path.join(working_directory, "ont_arg_related.list")
ont_arg_related_reads = os.path.join(working_directory, "ont_arg_related_reads.fasta")
ont_arg_mge_association = os.path.join(working_directory, "ont_arg_mge_blast.out")
ont_arg_related_reads_plasflow = os.path.join(working_directory, "ont_arg_related_reads_plasflow")
ont_arg_related_reads_plasflow_unclassified =f"{ont_arg_related_reads_plasflow}_unclassified.fasta"
ont_arg_related_reads_plasflow_unclassified_blast = os.path.join(working_directory, "ont_arg_related_reads_plasflow_unclassified_blast.out")
ont_plasmid_reads = os.path.join(working_directory, "ont_plasmid_reads.fasta")
ont_plasmid_reads_list_file = os.path.join(working_directory, "ont_plasmid_reads.list")
ont_mob_suite = os.path.join(working_directory, "ont_plasmid_mob_suite_analysis.out")
ont_abundance_report = os.path.join(working_directory, "ont_abundance.csv")
ont_centrifuge_classification_out = os.path.join(working_directory, "ont_centrifuge.out")
ont_centrifuge_classification_report = os.path.join(working_directory, "ont_centrifuge.report")
ont_arg_reads_best_hit = os.path.join(working_directory, "ont_arg_reads_best_hit.csv")
ont_chromosome_list_file = os.path.join(working_directory, "ont_chromosome.list")

bgi_contigs_folder = os.path.join(working_directory, "bgi.contig.fa")
bgi_contigs = os.path.join(bgi_contigs_folder, "bgi.contig.fa")
bgi_rgi_folder = os.path.join(working_directory, "rgi")
Path(bgi_rgi_folder).mkdir(parents=True, exist_ok=True)
bgi_rgi_report_prefix = os.path.join(bgi_rgi_folder, "rgi_output")
bgi_rgi_report = os.path.join(bgi_rgi_folder, "rgi_output.txt")
bgi_arg_related_reads_list = os.path.join(working_directory, "bgi_arg_related.list")
bgi_arg_related_contigs = os.path.join(working_directory, "bgi_arg_related_contigs.fasta")
bgi_arg_mge_association = os.path.join(working_directory, "bgi_arg_mge_blast.out")
bgi_arg_related_reads_plasflow = os.path.join(working_directory, "bgi_arg_related_reads_plasflow")
bgi_arg_related_reads_plasflow_unclassified = f"{bgi_arg_related_reads_plasflow}_unclassified.fasta"
bgi_arg_related_reads_plasflow_unclassified_blast = os.path.join(working_directory, "bgi_arg_related_reads_plasflow_unclassified_blast.out")
bgi_contig_regions_bed = os.path.join(working_directory, "bgi_contig_regions.bed")
bgi_bowtie2_index = os.path.join(bgi_contigs_folder, "bgi.contig")
bgi_bowtie2_output = os.path.join(bgi_contigs_folder, "bgi_read_contig.sam")
bgi_reads_contig_mapping = os.path.join(bgi_contigs_folder, "bgi_read_contig_sorted.bam")
bgi_total_contig_cov = os.path.join(working_directory, "bgi_total_contig.cov")
bgi_plasmid_reads_list_file = os.path.join(working_directory, "bgi_plasmid_reads.list")
bgi_plasmid_reads = os.path.join(working_directory, "bgi_plasmid_reads.fasta")
bgi_mob_suite = os.path.join(working_directory, "bgi_plasmid_mob_suite_analysis.out")
bgi_abundance_report = os.path.join(working_directory, "bgi_abundance.csv")
bgi_centrifuge_classification_out = os.path.join(working_directory, "bgi_centrifuge.out")
bgi_centrifuge_classification_report = os.path.join(working_directory, "bgi_centrifuge.report")
bgi_mge_reads_best_hit = os.path.join(working_directory, "bgi_arg_mge_best_hit.csv")
bgi_chromosome_list_file = os.path.join(working_directory, "bgi_chromosome.list")



# Functions


In [5]:
def run_card_blast(reads_loc, output_loc, blastdb=card_database):
    output_fmt = '"6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen"'
    cmd = f"blastn  -task megablast -num_threads {threads} -query {reads_loc} -outfmt {output_fmt} -db {blastdb} -out {output_loc}"
    print(f"running command {cmd}")
    os.system(cmd)

In [6]:
def run_seqtk_subseq(query_file, candidate_list, output_file):
    cmd = f"seqtk subseq {query_file} {candidate_list} > {output_file}"
    print(f"running command {cmd}")
    os.system(cmd)

In [7]:
def export_candidate_read_list(candidate_list_file, best_hit_list):
    with open(candidate_list_file, "w") as arg_related_f:
        best_hit_read_list = []
        for i in best_hit_list:
            best_hit_read_list.append(i[1])  # i[1]: read id
        for i in set(best_hit_read_list):
            arg_related_f.writelines(f"{i}\n")

In [8]:
def write_candidate_list_to_file(candidate_list_file, candidate_list):
    with open(candidate_list_file, "w") as arg_contig_list_f:
        for i in candidate_list:
            arg_contig_list_f.writelines(f"{i}\n")

In [9]:
def get_arg_mge_association(
    mge_best_hit, arg_best_hit, max_distance_range=[0, 100, 200, 500, 1000]
):
    mge_best_hit_df = pd.DataFrame(
        mge_best_hit,
        columns=[
            "index",
            "qseqid",
            "sseqid",
            "pident",
            "length",
            "mismatch",
            "gapopen",
            "qstart",
            "qend",
            "sstart",
            "send",
            "evalue",
            "bitscore",
            "slen",
            "qcov",
        ],
    )
    result_distance_dict = dict.fromkeys(max_distance_range)
    for k in max_distance_range:
        arg_mge_association_df = pd.DataFrame(columns=["ARG", "read_id", "mge_id"])
        for read_to_arg in arg_best_hit:
            arg_name = read_to_arg[2]
            seq_name = read_to_arg[1]
            seq_arg_qstart = min(read_to_arg[7], read_to_arg[8])
            seq_arg_qend = max(read_to_arg[7], read_to_arg[8])

            arg_related_mge_df = mge_best_hit_df[mge_best_hit_df["qseqid"] == seq_name]
            for j in arg_related_mge_df.iterrows():
                mge_align = j[1]
                mge_align_qstart = min(mge_align.qstart, mge_align.qend)
                mge_align_qend = max(mge_align.qstart, mge_align.qend)
                mge_type = mge_align.sseqid
                if (seq_arg_qstart <= mge_align_qend + k) and (
                    seq_arg_qend >= mge_align_qstart - k
                ):
                    arg_related_mge = mge_type
                    #                     print(arg_related_mge, arg_name, seq_name, seq_arg_qstart, seq_arg_qend, mge_align_qend, mge_align_qstart)
                    arg_mge_association_df = arg_mge_association_df.append(
                        {
                            "ARG": arg_name,
                            "read_id": seq_name,
                            "mge_id": arg_related_mge,
                        },
                        ignore_index=True,
                    )
        result_distance_dict[k] = arg_mge_association_df
    return result_distance_dict

In [10]:
def check_all_arg_mge_distance(
    mge_best_hit, arg_best_hit):
    mge_best_hit_df = pd.DataFrame(
        mge_best_hit,
        columns=[
            "index",
            "qseqid",
            "sseqid",
            "pident",
            "length",
            "mismatch",
            "gapopen",
            "qstart",
            "qend",
            "sstart",
            "send",
            "evalue",
            "bitscore",
            "slen",
            "qcov",
        ],
    )
    arg_mge_association_df = pd.DataFrame(columns=["ARG", "read_id", "mge_id", "distance", "mge_pident", "mge_lencov"])
    for read_to_arg in arg_best_hit:
        arg_name = read_to_arg[2]
        seq_name = read_to_arg[1]
        seq_arg_qstart = min(read_to_arg[7], read_to_arg[8])
        seq_arg_qend = max(read_to_arg[7], read_to_arg[8])
        seq_arg_middle = (seq_arg_qstart+seq_arg_qend)/2.0
        arg_related_mge_df = mge_best_hit_df[mge_best_hit_df["qseqid"] == seq_name]
        for j in arg_related_mge_df.iterrows():
            mge_align = j[1]
            mge_align_qstart = min(mge_align.qstart, mge_align.qend)
            mge_align_qend = max(mge_align.qstart, mge_align.qend)
            mge_align_middle = (mge_align_qstart+mge_align_qend)/2.0
            mge_type = mge_align.sseqid
            arg_related_mge = mge_type
            arg_mge_distance = None
            if seq_arg_qstart > mge_align_qstart:
                arg_mge_distance = seq_arg_qstart - mge_align_qend
            else:
                arg_mge_distance = mge_align_qstart -seq_arg_qend
            arg_mge_association_df = arg_mge_association_df.append(
                {
                    "ARG": arg_name,
                    "read_id": seq_name,
                    "mge_id": arg_related_mge,
                    "distance": arg_mge_distance,
                    "mge_pident": mge_align.pident,
                    "mge_lencov": mge_align.qcov,
                },
                ignore_index=True,
            )
        
    return arg_mge_association_df

In [11]:
def run_centrifuge(reads_loc, classification_output, report_file):
    cmd = f"centrifuge -x {centrifuge_database} -U {reads_loc} -f -S {classification_output} --report-file {report_file} -p {threads}"
    print(f"running command {cmd}")
    os.system(cmd)

In [12]:
def run_nt_blast(reads_loc, output_loc, blastdb=blast_nt_database):
    output_fmt = '"6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen stitle"'
    cmd = f"blastn  -task megablast -num_threads {threads} -query {reads_loc} -outfmt {output_fmt} -db {blastdb} -out {output_loc}"
    print(f"running command {cmd}")
    os.system(cmd)

In [13]:
def run_mge_blast(reads_loc, output_loc, blastdb=mge_database):
    output_fmt = '"6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen"'
    cmd = f"blastx -num_threads {threads} -query {reads_loc} -outfmt {output_fmt} -db {blastdb} -out {output_loc}"
    print(f"running command {cmd}")
    os.system(cmd)

In [14]:
def check_blast_alignment(blastn_card_result, threshold=0.7, overlap_borderline=20):
    """
    Given a sample's reads' blast result against card database, run quality filter and length filter.
    Make sure there is no overlap when there are multiple hits for one read, fuzzy the overlapping borderline by 20bps.
    Return:
        best_hit_list: this variable consists the best arg hit of each read in the database.
        Hits do not meet the lencov and pident requirements are discarded.
    """
    blast_output_fmt = [
        "qseqid",
        "sseqid",
        "pident",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
        "slen",
    ]
    blastn_card_df = pd.read_csv(
        blastn_card_result, sep="\t", header=None, names=blast_output_fmt
    )
    blastn_card_df_pident = blastn_card_df[blastn_card_df.pident >= threshold * 100]
    blastn_card_df_pident["lencov"] = (
        blastn_card_df_pident.length / blastn_card_df_pident.slen
    )
    blastn_card_df_lencov = blastn_card_df_pident[
        blastn_card_df_pident.lencov >= threshold
    ]
    seqid_list = list(set(list(blastn_card_df_lencov.qseqid)))
    best_hit_list = []
    for seqid in seqid_list:
        seq_df = blastn_card_df_lencov[
            blastn_card_df_lencov.qseqid == seqid
        ].sort_values(by=["pident", "lencov"], ascending=[False, False])
        seq_df.reset_index(inplace=True)
        seq_df_values = seq_df.values
        seq_alignment_block = []
        for i in seq_df_values:
            qstart = min(i[7], i[8])  # sometimes there are reverse ones
            qend = max(i[7], i[8])
            append_flag = 1
            for j in seq_alignment_block:
                if (qend >= j[0] + overlap_borderline) and (
                    qstart <= j[1] - overlap_borderline
                ):
                    j[2].append([i])
                    j[0] = min(qstart, j[1])
                    j[1] = max(qend, j[0])
                    append_flag = 0
            if append_flag == 1:
                seq_alignment_block.append([qstart, qend, [i]])
        for i in seq_alignment_block:
            best_hit_list.append(i[2][0])
    return best_hit_list

In [15]:
def get_blast_names(megablast_result, blast_cutoff=5):
    """
    Given a blast result against nt database, this function return top 5 hits with stitle of blast result in the database
    """
    reads_list = []
    nt_blast_output_fmt = [
        "qseqid",
        "sseqid",
        "pident",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
        "slen",
        "stitle",
    ]
    megablast_df = pd.read_csv(
        megablast_result, sep="\t", header=None, names=nt_blast_output_fmt
    )
    #     for record in SeqIO.parse(read_file, "fasta"):
    #         reads_list.append(record.id)
    reads_blast_dict = {}
    for read_id in list(set(list(megablast_df.qseqid))):
        #         if read_id in list(megablast_df.qseqid):
        cutoff_df = megablast_df.groupby("qseqid").get_group(read_id)[:blast_cutoff]
        reads_blast_dict.update({read_id: cutoff_df.stitle.values})
    return reads_blast_dict

In [16]:
def get_read_location_from_blast(blast_result_dict):
    location_dict = {}
    for read_id in blast_result_dict.keys():
        first_hit = blast_result_dict[read_id][0]
        if "plasmid" in first_hit.split(" "):
            location_dict.update({read_id: "plasmid"})
        elif "chromosome" in first_hit.split(" "):
            location_dict.update({read_id: "chromosome"})
        else:
            location_dict.update({read_id: "unclassified"})
    return location_dict

In [17]:
def get_short_read_cov(
    bgi_rgi_output,
    region_bed,
    total_contig_cov,
    short_read_mapping,
    samtools_quality_filter=20,
):
    # /home/project_data/wastewater_arg/bgi/inf_reads/mapping/inf_read_contig_sorted.bam
    with open(region_bed, "w") as rgi_alignment_region:
        for i in bgi_rgi_output.iterrows():
            content = i[1]
            ctg_split = content["Contig"].split("_")
            contig_name = f"{ctg_split[0]}_{ctg_split[1]}"
            #             contig_name = content["Contig"]
            start = content["Start"]
            stop = content["Stop"]
            rgi_alignment_region.writelines(f"{contig_name}\t{start}\t{stop}\n")
    samtools_cmd = f"samtools bedcov -j -Q {samtools_quality_filter} {region_bed} {short_read_mapping} > {total_contig_cov}"
    print(f"running command {samtools_cmd}")
    os.system(samtools_cmd)
    ctg_base_count_df = pd.read_csv(
        total_contig_cov,
        sep="\t",
        header=None,
        names=["ctg_name", "start", "stop", "total_base_count"],
    )
    ctg_base_count_df["avg_base_count"] = ctg_base_count_df["total_base_count"] / (
        ctg_base_count_df["stop"] - ctg_base_count_df["start"]
    )
    ctg_base_count_df["ctg_name_id"] = bgi_rgi_output.reset_index().Contig
    return ctg_base_count_df

In [18]:
def rgi_output_filter(file_loc, identity_threshold=0.7, length_threshold=0.7):
    """
    Quality filter for rgi results.
    Return: a dataframe consists rgi result with qualified reads
    """
    i_t = identity_threshold * 100
    l_t = length_threshold * 100
    rgi_df = pd.read_csv(file_loc, sep="\t")
    df_after_i_t = rgi_df[rgi_df.Best_Identities >= i_t]
    df_after_l_t = df_after_i_t[
        df_after_i_t["Percentage Length of Reference Sequence"] >= l_t
    ]
    return df_after_l_t

In [19]:
def get_rgi_contig_list(rgi_output_df):
    """
    given a filtered rgi output, return a list of contigs that is associated with args
    """
    contig_name_list = set()
    for i in rgi_output_df.Contig:
        contig_name_l = i.split("_")
        contig_name = "_".join(contig_name_l[:2])
        contig_name_list.add(contig_name)
    return list(contig_name_list)

In [20]:
def get_location_and_abundance(
    arg_related_reads_plasflow,
    arg_best_hit,
    mge_best_hit,
    arg_related_reads_plasflow_unclassified_blast_location_dict,
    lr=True,
    sr_avg_read_cov=None,
):
    chromosome_list = []
    best_hit_columns = [
        "index",
        "qseqid",
        "sseqid",
        "pident",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
        "slen",
        "qcov",
    ]
    # PREPROCESS
    # read dataframe and csv
    arg_related_reads_plasflow_df = pd.read_csv(arg_related_reads_plasflow, sep="\t")
    mge_best_hit_df = pd.DataFrame(mge_best_hit, columns=best_hit_columns)[
        ["qseqid", "sseqid"]
    ]
    # get arg name to read id dict
    # get arg name to arg acc dict
    if lr:
        arg_best_hit_df = pd.DataFrame(arg_best_hit, columns=best_hit_columns)[
            ["qseqid", "sseqid"]
        ]
        arg_best_hit_df["arg_acc"] = arg_best_hit_df["sseqid"].apply(extract_arg_acc)
        arg_best_hit_df["arg_name"] = arg_best_hit_df["arg_acc"].apply(extract_arg)
        arg_reads_dict = dataframe_to_dict(arg_best_hit_df, "arg_name", "qseqid")
        arg_acc_name_dict = dict(zip(arg_best_hit_df.arg_name, arg_best_hit_df.arg_acc))
    else:
        arg_best_hit_df = arg_best_hit
        arg_best_hit_df["contig_name"] = arg_best_hit_df["Contig"].apply(extract_contig)
        arg_best_hit_df["arg_acc"] = arg_best_hit_df["ARO"].apply(add_ARO)
        arg_reads_dict = dataframe_to_dict(
            arg_best_hit_df, "Best_Hit_ARO", "contig_name"
        )
        arg_reads_id_dict = dataframe_to_dict(arg_best_hit_df, "Best_Hit_ARO", "Contig")
        arg_acc_name_dict = dict(
            zip(arg_best_hit_df.Best_Hit_ARO, arg_best_hit_df.arg_acc)
        )
    # get read id to mge type dict
    mge_best_hit_df["mge_type"] = mge_best_hit_df["sseqid"].apply(extract_mge)
    mge_reads_dict = dataframe_to_dict(mge_best_hit_df, "qseqid", "mge_type")
    # get read id to plasflow location dict
    plawflow_df_info = arg_related_reads_plasflow_df[["contig_name", "label"]]
    plawflow_df_info["label"] = plawflow_df_info["label"].apply(extract_plasflow)
    read_plasflow_dict = dict(zip(plawflow_df_info.contig_name, plawflow_df_info.label))

    # GENERATE RESULTS
    # result dataframe
    stats_column = [
        "transposase",
        "IntI1",
        "integrase",
        "recombinase",
        "plasmid",
        "chromosome",
        "unclassified",
        "arg_name",
        "arg_ref_len",
        "arg_read_num",
    ]
    abundance_df = pd.DataFrame(columns=stats_column)

    result_dict = {}
    for arg_name in arg_reads_dict.keys():
        transposase_num = 0
        IntI1_num = 0
        integrase_num = 0
        recombinase_num = 0
        plasmid_num = 0
        chromosome_num = 0
        unclassified_num = 0
        read_list = arg_reads_dict[arg_name]
        if lr:
            read_num = len(read_list)
        else:
            read_id_list = arg_reads_id_dict[arg_name]
            sr_cov_dict = dict(
                zip(sr_avg_read_cov.ctg_name_id, sr_avg_read_cov.avg_base_count)
            )
            read_num = 0
            for contig in read_id_list:
                read_num += sr_cov_dict[contig]
        arg_len = aro_len_dict[arg_acc_name_dict[arg_name]]
        # plasflow location and blast location
        for read_id in read_list:
            plasflow_location = read_plasflow_dict[read_id]
            if plasflow_location == "plasmid":
                plasmid_num += 1
            elif plasflow_location == "chromosome":
                chromosome_list.append(read_id)
                chromosome_num += 1
            else:
                if (
                    read_id
                    in arg_related_reads_plasflow_unclassified_blast_location_dict.keys()
                ):
                    blast_location = (
                        arg_related_reads_plasflow_unclassified_blast_location_dict[
                            read_id
                        ]
                    )
                    if blast_location == "plasmid":
                        plasmid_num += 1
                    elif blast_location == "chromosome":
                        chromosome_list.append(read_id)
                        chromosome_num += 1
                    else:
                        unclassified_num += 1
                else:
                    unclassified_num += 1
            # mges
            if read_id in mge_reads_dict.keys():
                mge_list = mge_reads_dict[read_id]
                for mge_type in mge_list:
                    if mge_type == "transposase":
                        transposase_num += 1
                    elif mge_type == "IntI1":
                        IntI1_num += 1
                    elif mge_type == "integrase":
                        integrase_num += 1
                    elif mge_type == "recombinase":
                        recombinase_num += 1

        abundance_df = abundance_df.append(
            {
                "transposase": transposase_num,
                "IntI1": IntI1_num,
                "integrase": integrase_num,
                "recombinase": recombinase_num,
                "plasmid": plasmid_num,
                "chromosome": chromosome_num,
                "unclassified": unclassified_num,
                "arg_name": arg_name,
                "arg_ref_len": arg_len,
                "arg_read_num": read_num,
            },
            ignore_index=True,
        )
    return abundance_df, chromosome_list

In [21]:
def extract_plasmid_read_id_list(
    arg_related_reads_plasflow,
    arg_related_reads_plasflow_unclassified_blast_location_dict,
):
    plasflow_df = pd.read_csv(arg_related_reads_plasflow, sep="\t")
    plawflow_df_info = plasflow_df[["contig_name", "label"]]
    plawflow_df_info["label"] = plawflow_df_info["label"].apply(extract_plasflow)
    read_plasflow_dict = dict(zip(plawflow_df_info.contig_name, plawflow_df_info.label))
    plasmid_read_list = []
    for i in read_plasflow_dict:
        if read_plasflow_dict[i] == "plasmid":
            plasmid_read_list.append(i)
        elif read_plasflow_dict[i] == "unclassified":
            if i in list(
                arg_related_reads_plasflow_unclassified_blast_location_dict.keys()
            ):
                if (
                    arg_related_reads_plasflow_unclassified_blast_location_dict[i]
                    == "plasmid"
                ):
                    plasmid_read_list.append(i)
    return plasmid_read_list

In [22]:
def run_mob_suite(in_file, out_file):
    cmd = f"mob_typer -i {os.path.abspath(in_file)} -o {os.path.abspath(out_file)} -n {threads} -x -f"
    print(f"{cmd}")


#     os.system(cmd)

In [23]:
def extract_mge(mge_str):
    return mge_str.split("|")[-2]


def extract_arg(arg_acc):
    aro_name = aro_acc_name_dict[arg_acc].split("[")[0][:-1]
    return aro_name


def extract_arg_acc(mge_str):
    return mge_str.split("|")[-2]


def extract_plasflow(mge_str):
    return mge_str.split(".")[0]


def extract_contig(mge_str):
    contig_l = mge_str.split("_")
    return f"{contig_l[0]}_{contig_l[1]}"


def add_ARO(aro_num):
    return f"ARO:{aro_num}"

In [24]:
def output_best_hit(arg_best_hit, output_csv):
    best_hit_columns = [
        "index",
        "qseqid",
        "sseqid",
        "pident",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
        "slen",
        "qcov",
    ]
    arg_best_hit_df = pd.DataFrame(arg_best_hit, columns=best_hit_columns)
    arg_best_hit_df.to_csv(output_csv)

In [25]:
def dataframe_to_dict(transform_df, column_1, column_2):
    """
    Transform a dataframe into dictionary. column 1 is the key, column 2 is the value.
    In this function, value can be multiple ones, so they are stored in a list.
    """
    ret_dict = {}
    for data_series in transform_df.iterrows():
        row_content = data_series[1]
        if row_content[column_1] in list(ret_dict.keys()):
            ret_dict[row_content[column_1]].append(row_content[column_2])
        else:
            ret_dict.update({row_content[column_1]: [row_content[column_2]]})
    return ret_dict

In [26]:
def write_list_to_file(writing_list, writing_file):
    with open(writing_file, "w") as f:
        for i in writing_list:
            f.writelines(f"{i}\n")

# ARG identification with CARD database

In [110]:
"""
Parse CARD database, link ARO acc to its length and its name
"""
aro_len_dict = {}
aro_acc_name_dict = {}
with open(card_database) as card_db:
    for record in SeqIO.parse(card_db, "fasta"):
        record_id_list = record.description.split("|")
        # print(record_id_list)
        record_acc = record_id_list[4]
        record_name = record_id_list[5]
        record_len = len(record.seq)
        aro_len_dict.update({record_acc: record_len})
        aro_acc_name_dict.update({record_acc: record_name})
with open(card_database_general) as card_db:
    for record in SeqIO.parse(card_db, "fasta"):
        record_id_list = record.description.split("|")
        record_acc = record_id_list[0]
        #         record_name = record_id_list[5]
        record_len = len(record.seq)
        aro_len_dict.update({record_acc: record_len})
#         aro_acc_name_dict.update({record_acc:record_name})

# Short read assembly

In [34]:
os.system(f"megahit -1 {short_reads_1_location} -2 {short_reads_2_location} -t {threads} -o {bgi_contigs}")

2022-05-03 15:18:44 - MEGAHIT v1.2.9
2022-05-03 15:18:44 - Using megahit_core with POPCNT and BMI2 support
2022-05-03 15:18:44 - Convert reads to binary library
2022-05-03 15:19:27 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/scratch0/yf20/ww_sample_comparison_2/B_ww_2/B_ww_2_Illumina_1.fastq,/scratch0/yf20/ww_sample_comparison_2/B_ww_2/B_ww_2_Illumina_2.fastq): pe, 37342672 reads, 151 max length'
2022-05-03 15:19:27 - b'INFO  utils/utils.h                 :  152 - Real: 43.9087\tuser: 34.1259\tsys: 6.0477\tmaxrss: 254280'
2022-05-03 15:19:27 - k-max reset to: 141 
2022-05-03 15:19:27 - Start assembly. Number of CPU threads 30 
2022-05-03 15:19:27 - k list: 21,29,39,59,79,99,119,141 
2022-05-03 15:19:27 - Memory used: 962179802726
2022-05-03 15:19:27 - Extract solid (k+1)-mers for k = 21 
2022-05-03 15:20:53 - Build graph for k = 21 
2022-05-03 15:22:10 - Assemble contigs from SdBG for k = 21
2022-05-03 15:31:32 - Local assembly for k = 21
2022-05-03 15:33:00 - Extract iter

0

In [127]:
# remove contigs less than 500 bps
contig_seq = os.path.join(bgi_contigs_folder, "final.contigs.fa")
os.system(f"seqkit seq -m 500 {contig_seq} > {bgi_contigs}")

[WARN] you may switch on flag -g/--remove-gaps to remove spaces


0

## ONT fasta reads

In [128]:
# run blast against CARD database for inf, eff and fin reads
run_card_blast(ont_reads, ont_card_blast)

running command blastn  -task megablast -num_threads 30 -query /scratch0/yf20/ww_sample_comparison_2/B_ww_3/B_ww_3_ont.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen" -db /home/Users/yf20/databases/card/card-data/nucleotide_fasta_protein_homolog_model.fasta -out /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_card_blast.out


In [129]:
# get the best hit of all qualified reads-arg pairs
ont_arg_best_hit = check_blast_alignment(ont_card_blast)

In [130]:
output_best_hit(ont_arg_best_hit, ont_arg_reads_best_hit)

In [131]:
# export arg-related reads as a list
export_candidate_read_list(ont_arg_related_reads_list, ont_arg_best_hit)

In [132]:
# extract ONT arg related reads
run_seqtk_subseq(ont_reads, ont_arg_related_reads_list, ont_arg_related_reads)

running command seqtk subseq /scratch0/yf20/ww_sample_comparison_2/B_ww_3/B_ww_3_ont.fasta /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related.list > /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads.fasta


## short reads

In [134]:
os.system(f"rgi main -i {bgi_contigs} -o {bgi_rgi_report_prefix} -n {threads}")

/home/Users/yf20/miniconda3/lib/python3.8/site-packages/requests-2.24.0-py3.8.egg/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


0

In [138]:
# get filtered rgi results
rgi_filtered = rgi_output_filter(bgi_rgi_report)

In [139]:
# get arg-related contigs as a list
rgi_contig_list = get_rgi_contig_list(rgi_filtered)

In [140]:
# write arg-related contigs list to file
write_candidate_list_to_file(bgi_arg_related_reads_list, rgi_contig_list)

In [141]:
# extract BGI arg related contigs
run_seqtk_subseq(bgi_contigs, bgi_arg_related_reads_list, bgi_arg_related_contigs)

running command seqtk subseq /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi.contig.fa /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related.list > /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_contigs.fasta


# ARG-associated reads MGE identification

## ONT reads

In [142]:
# run blast against mge reads for arg-related inf, eff and fin reads
run_mge_blast(ont_arg_related_reads, ont_arg_mge_association)

running command blastx -num_threads 30 -query /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen" -db /home/Users/yf20/databases/mge/MGEs90.fasta -out /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_mge_blast.out


In [143]:
# get each read' best mge hits
ont_mge_best_hit = check_blast_alignment(ont_arg_mge_association)

In [148]:
# generate a dictionary that have arg and mge within a certain distance
ont_arg_mge_distance_range = get_arg_mge_association(ont_mge_best_hit, ont_arg_best_hit)

In [149]:
check_all_arg_mge_distance(ont_mge_best_hit, ont_arg_best_hit).to_csv(os.path.join(working_directory,"ont_arg_mge_distance.csv"))

In [150]:
for range_num in ont_arg_mge_distance_range:
    ont_arg_mge_distance_range[range_num].to_csv(os.path.join(working_directory, f"inf_arg_mge_association_{range_num}.csv"))

## Short reads

In [151]:
# run blast against mge reads for arg-related inf, eff and fin reads
run_mge_blast(bgi_arg_related_contigs, bgi_arg_mge_association)

running command blastx -num_threads 30 -query /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_contigs.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen" -db /home/Users/yf20/databases/mge/MGEs90.fasta -out /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_mge_blast.out


In [152]:
# get each contig' best mge hits
bgi_mge_best_hit = check_blast_alignment(bgi_arg_mge_association)

In [153]:
output_best_hit(bgi_mge_best_hit, bgi_mge_reads_best_hit)

#### RGI does not have blast-like output for locations, so cannot determine if a ARG is flanked by MGE

## PlasFlow analysis

### PlasFlow has to be run at another environment, here are commands for copy-pasting:
## ONT reads

In [154]:
print("conda activate plasflow")
print(f"PlasFlow.py --input {os.path.abspath(ont_arg_related_reads)} --output {os.path.abspath(ont_arg_related_reads_plasflow)}")
print("conda activate base")

conda activate plasflow
PlasFlow.py --input /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads.fasta --output /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads_plasflow
conda activate base


## Short reads

In [155]:
print("conda activate plasflow")
print(f"PlasFlow.py --input {os.path.abspath(bgi_arg_related_contigs)} --output {os.path.abspath(bgi_arg_related_reads_plasflow)}")
print("conda activate base\n")

conda activate plasflow
PlasFlow.py --input /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_contigs.fasta --output /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_reads_plasflow
conda activate base



## Classify PlasFlow unclassified reads with blastn

## ONT reads

In [156]:
# use megablast to check unclassified results from plasflow
run_nt_blast(ont_arg_related_reads_plasflow_unclassified, ont_arg_related_reads_plasflow_unclassified_blast,)

running command blastn  -task megablast -num_threads 30 -query /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads_plasflow_unclassified.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen stitle" -db /home/dbs/SeqScreenDB/blast/nt/nt -out /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads_plasflow_unclassified_blast.out


## Short reads

In [157]:
# use megablast to check unclassified results from plasflow
run_nt_blast(bgi_arg_related_reads_plasflow_unclassified, bgi_arg_related_reads_plasflow_unclassified_blast,)

running command blastn  -task megablast -num_threads 30 -query /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_reads_plasflow_unclassified.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore slen stitle" -db /home/dbs/SeqScreenDB/blast/nt/nt -out /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_reads_plasflow_unclassified_blast.out


In [158]:
ont_arg_related_reads_plasflow_unclassified_blast_dict = get_blast_names(ont_arg_related_reads_plasflow_unclassified_blast)

ont_arg_related_reads_plasflow_unclassified_blast_location_dict = (get_read_location_from_blast(ont_arg_related_reads_plasflow_unclassified_blast_dict))

In [159]:
bgi_arg_related_reads_plasflow_unclassified_blast_dict = get_blast_names(bgi_arg_related_reads_plasflow_unclassified_blast)

bgi_arg_related_reads_plasflow_unclassified_blast_location_dict = (get_read_location_from_blast(bgi_arg_related_reads_plasflow_unclassified_blast_dict))

# Combine Plasflow, blast and MGE information. Add abundance calculation.

In [160]:
# map reads to contigs
subprocess.run(["bowtie2-build", bgi_contigs, bgi_bowtie2_index], stdout=subprocess.DEVNULL)

Building a SMALL index


CompletedProcess(args=['bowtie2-build', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi.contig.fa', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi.contig'], returncode=0)

In [164]:
subprocess.run(["bowtie2", "-x", bgi_bowtie2_index, "-1", short_reads_1_location, "-2",  short_reads_2_location, "-S",  bgi_bowtie2_output, "-p", str(threads)])

17103099 reads; of these:
  17103099 (100.00%) were paired; of these:
    11229508 (65.66%) aligned concordantly 0 times
    4196673 (24.54%) aligned concordantly exactly 1 time
    1676918 (9.80%) aligned concordantly >1 times
    ----
    11229508 pairs aligned concordantly 0 times; of these:
      1729140 (15.40%) aligned discordantly 1 time
    ----
    9500368 pairs aligned 0 times concordantly or discordantly; of these:
      19000736 mates make up the pairs; of these:
        13691747 (72.06%) aligned 0 times
        1939442 (10.21%) aligned exactly 1 time
        3369547 (17.73%) aligned >1 times
59.97% overall alignment rate


CompletedProcess(args=['bowtie2', '-x', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi.contig', '-1', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/B_ww_3_Illumina_1.fastq', '-2', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/B_ww_3_Illumina_2.fastq', '-S', '/scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi_read_contig.sam', '-p', '30'], returncode=0)

In [165]:
pysam.sort("-@", str(threads), "-o", bgi_reads_contig_mapping, bgi_bowtie2_output)

''

In [166]:
pysam.index(bgi_reads_contig_mapping)

''

In [167]:
# get short read contigs average per base coverage
bgi_read_cov = get_short_read_cov(
    rgi_filtered,
    bgi_contig_regions_bed,
    bgi_total_contig_cov,
    bgi_reads_contig_mapping,
)

running command samtools bedcov -j -Q 20 /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_contig_regions.bed /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi_read_contig_sorted.bam > /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_total_contig.cov


In [168]:
ont_loc_abundance_report, ont_chromosome_list = get_location_and_abundance(
    ont_arg_related_reads_plasflow,
    ont_arg_best_hit,
    ont_mge_best_hit,
    ont_arg_related_reads_plasflow_unclassified_blast_location_dict,
)

In [169]:
write_list_to_file(ont_chromosome_list, ont_chromosome_list_file)     

In [170]:
ont_loc_abundance_report.to_csv(ont_abundance_report)

In [171]:
bgi_loc_abundance_report, bgi_chromosome_list = get_location_and_abundance(
    bgi_arg_related_reads_plasflow,
    rgi_filtered,
    bgi_mge_best_hit,
    bgi_arg_related_reads_plasflow_unclassified_blast_location_dict,
    lr=False,
    sr_avg_read_cov=bgi_read_cov,
)

In [172]:
write_list_to_file(bgi_chromosome_list, bgi_chromosome_list_file) 

In [173]:
bgi_loc_abundance_report.to_csv(bgi_abundance_report)

# MOB-suite for plasmid annotation

In [174]:
ont_plasmid_read_list = extract_plasmid_read_id_list(
    ont_arg_related_reads_plasflow,
    ont_arg_related_reads_plasflow_unclassified_blast_location_dict,
)

In [175]:
bgi_plasmid_read_list = extract_plasmid_read_id_list(
    bgi_arg_related_reads_plasflow,
    bgi_arg_related_reads_plasflow_unclassified_blast_location_dict,
)

In [176]:
write_candidate_list_to_file(ont_plasmid_reads_list_file, ont_plasmid_read_list)

In [177]:
write_candidate_list_to_file(bgi_plasmid_reads_list_file, bgi_plasmid_read_list)

In [178]:
# extract ONT plasmid reads
run_seqtk_subseq(ont_reads, ont_plasmid_reads_list_file, ont_plasmid_reads)

running command seqtk subseq /scratch0/yf20/ww_sample_comparison_2/B_ww_3/B_ww_3_ont.fasta /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_plasmid_reads.list > /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_plasmid_reads.fasta


In [179]:
# extract short bgi plasmid reads
run_seqtk_subseq(bgi_contigs, bgi_plasmid_reads_list_file, bgi_plasmid_reads)

running command seqtk subseq /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi.contig.fa/bgi.contig.fa /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_plasmid_reads.list > /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_plasmid_reads.fasta


In [180]:
# run mob suite
# NEED TO RUN MOB INIT ON COMMAND LINE FIRST
os.system(f"mob_typer -i {os.path.abspath(ont_plasmid_reads)} -o {os.path.abspath(ont_mob_suite)} -n {threads} -x -f")

2022-05-04 14:05:17,546 mob_suite.mob_typer INFO: Running Mob-typer version 3.0.3 [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/mob_typer.py:163]
2022-05-04 14:05:17,546 mob_suite.mob_typer INFO: Processing fasta file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_plasmid_reads.fasta [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/mob_typer.py:165]
2022-05-04 14:05:17,547 mob_suite.mob_typer INFO: SUCCESS: Found program blastn at /home/Users/yf20/miniconda3/bin/blastn [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/utils.py:590]
2022-05-04 14:05:17,548 mob_suite.mob_typer INFO: SUCCESS: Found program makeblastdb at /home/Users/yf20/miniconda3/bin/makeblastdb [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/utils.py:590]
2022-05-04 14:05:17,548 mob_suite.mob_typer INFO: SUCCESS: Found program tblastn at /home/Users/yf20/miniconda3/bin/tblastn [in /home/Users/yf20/miniconda3/lib/python3.8/site-pack

0

In [181]:
os.system(f"mob_typer -i {os.path.abspath(bgi_plasmid_reads)} -o {os.path.abspath(bgi_mob_suite)} -n {threads} -x -f")
# run_mob_suite(bgi_plasmid_reads, bgi_mob_suite)

2022-05-04 14:09:38,892 mob_suite.mob_typer INFO: Running Mob-typer version 3.0.3 [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/mob_typer.py:163]
2022-05-04 14:09:38,892 mob_suite.mob_typer INFO: Processing fasta file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_plasmid_reads.fasta [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/mob_typer.py:165]
2022-05-04 14:09:38,893 mob_suite.mob_typer INFO: SUCCESS: Found program blastn at /home/Users/yf20/miniconda3/bin/blastn [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/utils.py:590]
2022-05-04 14:09:38,893 mob_suite.mob_typer INFO: SUCCESS: Found program makeblastdb at /home/Users/yf20/miniconda3/bin/makeblastdb [in /home/Users/yf20/miniconda3/lib/python3.8/site-packages/mob_suite/utils.py:590]
2022-05-04 14:09:38,893 mob_suite.mob_typer INFO: SUCCESS: Found program tblastn at /home/Users/yf20/miniconda3/bin/tblastn [in /home/Users/yf20/miniconda3/lib/python3.8/site-pack

0

# Centrifuge for classification

In [182]:
run_centrifuge(
    ont_arg_related_reads,
    ont_centrifuge_classification_out,
    ont_centrifuge_classification_report,
)

running command centrifuge -x /home/dbs/SeqScreenDB/centrifuge/abv -U /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_arg_related_reads.fasta -f -S /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_centrifuge.out --report-file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_centrifuge.report -p 30


report file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/ont_centrifuge.report
Number of iterations in EM algorithm: 0
Probability diff. (P - P_prev) in the last iteration: 0
Calculating abundance: 00:00:00


In [183]:
run_centrifuge(
    bgi_arg_related_contigs,
    bgi_centrifuge_classification_out,
    bgi_centrifuge_classification_report,
)

running command centrifuge -x /home/dbs/SeqScreenDB/centrifuge/abv -U /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_arg_related_contigs.fasta -f -S /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_centrifuge.out --report-file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_centrifuge.report -p 30


report file /scratch0/yf20/ww_sample_comparison_2/B_ww_3/bgi_centrifuge.report
Number of iterations in EM algorithm: 1
Probability diff. (P - P_prev) in the last iteration: 0
Calculating abundance: 00:00:00
